In [ ]:
from semantic_kernel import __version__
import os
from dotenv import load_dotenv
# Load environment variables from the .env file
load_dotenv()

__version__

In [ ]:
from datetime import datetime
import os
now = datetime.now()
output_dir = f"./output/{now.strftime('%Y_%m_%d_%H_%M_%S')}"
os.makedirs(output_dir, exist_ok=True)

In [ ]:
from semantic_kernel import Kernel

kernel = Kernel()

In [ ]:
from services import Service

from service_settings import ServiceSettings

service_settings = ServiceSettings()

# Select a service to use for this notebook (available services: OpenAI, AzureOpenAI, HuggingFace)
selectedService = (
    Service.AzureOpenAI
    if service_settings.global_llm_service is None
    else Service(service_settings.global_llm_service.lower())
)
print(f"Using service type: {selectedService}")

In [ ]:
from openai import AsyncAzureOpenAI
from image_gen_text_to_image import ImageGenTextToImage

image_client = AsyncAzureOpenAI(
    azure_endpoint=os.getenv("AZURE_IMAGE_GEN_ENDPOINT"),
    api_key=os.getenv("AZURE_IMAGE_GEN_API_KEY"),
    api_version=os.getenv("AZURE_IMAGE_GEN_API_VERSION"),
)   

# Remove all services so that this cell can be re-run without restarting the kernel
kernel.remove_all_services()

service_id = None
if selectedService == Service.OpenAI:
    from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion

    service_id = "default"
    kernel.add_service(
        OpenAIChatCompletion(
            service_id=service_id,
        ),
    )
elif selectedService == Service.AzureOpenAI:
    from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion

    service_id = "default"
    kernel.add_service(
        AzureChatCompletion(
            service_id=service_id,
        ),
    )

    from semantic_kernel.connectors.ai.open_ai import OpenAITextToImage 

    image_gen_service = ImageGenTextToImage (
        async_client=image_client,
        ai_model_id="gpt-image-1",
        service_id="gpt-image-1", # Optional; for targeting specific services within Semantic Kernel
    )
    kernel.add_service(image_gen_service)

In [ ]:
plugin = kernel.add_plugin(parent_directory=".", plugin_name="IdeaGenPlugin")

In [ ]:
import base64

# Function to convert a file to Base64
def file_to_base64(file_path):
    with open(file_path, "rb") as file:
        # Read the file in binary mode and encode it to Base64
        base64_encoded = base64.b64encode(file.read()).decode("utf-8")
    return base64_encoded

# Example usage
file_path = "image.png"  # Replace with your file path
base64_string = file_to_base64(file_path)


In [ ]:
image_gen_service = kernel.get_service(service_id="gpt-image-1")

### Generate the Idea

In [ ]:
from semantic_kernel.connectors.ai.open_ai import OpenAIChatPromptExecutionSettings
from semantic_kernel.contents.chat_history import ChatHistory
from semantic_kernel.contents import ChatMessageContent, TextContent, ImageContent
from semantic_kernel.contents.utils.author_role import AuthorRole
import time

seed = int(time.time())
print(f"Seed: {seed}")
execution_settings = OpenAIChatPromptExecutionSettings(seed=seed)

text_gen_service = kernel.get_service(service_id="default")

chat_history = ChatHistory()

input = "animal"

chat_history.add_message(ChatMessageContent(
        role=AuthorRole.USER,
        items=[
            TextContent(text=f"""GENERATE A ONE-SENTENCE IDEA FOR A PRODUCT THAT CAN BE 3D PRINTED BY A RASIN 3D PRINTER.  
            IT SHOULD BE A USEFUL PRODUCT LIKE A TOOTHBRUSH HOLDER OR A HOOD ORNAMENT.  IT SHOULD BE FUNNY, CREATIVE AND WIMSICAL.

BE CREATIVE AND FUNNY. I WANT TO LAUGH.
Incorporate the style suggestion, if provided: {{$style}}
+++++

{input}
+++++
                                """),
#            ImageContent(uri=f"data:image/png;base64,{file_to_base64('./image.png')}")
       ]
    ))

# Get the chat completion response
idea = await text_gen_service.get_chat_message_content(
    chat_history=chat_history,
    settings=execution_settings,
)

idea = str(idea)
print(idea)

### Generate Base Image

In [ ]:
print_idea = f"""Generate a photo of a plastic 3D-printed object based on the following idea: {idea}
The photo should be in a realistic style, with a white background and no text.
The object should be the only thing in the image, and it should be centered in the frame.
The object should be a 3D-printable object, and it should be in a realistic style.
The image must not be a drawing, painting, or cartoon.
"""

image = await image_gen_service.generate_image(
        description=print_idea, width=1024, height=1024, quality="auto"
    )

image_data = base64.b64decode(image)


base_image_path = f"{output_dir}/image.png"
with open(base_image_path, "wb") as image_file:
    image_file.write(image_data)


In [ ]:
from IPython.display import Image
Image(filename=base_image_path, width=300)

### Evaluate the image

In [ ]:
from semantic_kernel.connectors.ai.open_ai import OpenAIChatPromptExecutionSettings
from semantic_kernel.contents.chat_history import ChatHistory
from semantic_kernel.contents import ChatMessageContent, TextContent, ImageContent
from semantic_kernel.contents.utils.author_role import AuthorRole

execution_settings = OpenAIChatPromptExecutionSettings()

text_gen_service = kernel.get_service(service_id="default")

chat_history = ChatHistory()

base64_string = file_to_base64(f"./{output_dir}/image.png")

# prompt = """Evaluate the following image from the point of view of marketability and ease of 3D printing.  
#                                 Make a one-sentense improvement suggestion.
#                                 Only respond with the suggestion.  Do not explain the reasons for the suggestion.
#                                 """

prompt = f"""I'm preparing this image for a 3D printing project. What objects should be removed from the image to make it more suitable for 3D printing?""" 

chat_history.add_message(ChatMessageContent(
        role=AuthorRole.USER,
        items=[
            TextContent(text=prompt),
            ImageContent(uri=f"data:image/png;base64,{base64_string}")
        ]
    ))

# Get the chat completion response
eval_response = await text_gen_service.get_chat_message_content(
    chat_history=chat_history,
    settings=execution_settings,
)

eval_response = str(eval_response)
print(eval_response)

### Update the Image

In [ ]:
import requests

def edit_image(image, mask, prompt):
    url = f"{os.getenv('AZURE_IMAGE_GEN_ENDPOINT')}/openai/deployments/{os.getenv('AZURE_IMAGE_GEN_DEPLOYMENT_NAME')}/images/edits?api-version={os.getenv('AZURE_IMAGE_GEN_API_VERSION')}"

    headers = {
        "Authorization": f"Bearer {os.getenv('AZURE_IMAGE_GEN_API_KEY')}"
    }

    files = {
        'image': ('image.png', image),
        'mask': ('image.png', mask),
    }
    
    
    data = {
        "prompt": prompt
    }

    response = requests.post(url, data=data, files=files, headers=headers)

    print("Status Code:", response.status_code)
    with open("response.json", "w") as f:
        f.write(response.text)

    response_js = response.json()
    return response_js["data"][0]["b64_json"]



image_base_64 = edit_image(open(f"{output_dir}/image.png", "rb").read(), open("mask2.png", "rb").read(), eval_response)  


image_data = base64.b64decode(image_base_64)


iter_image_path = f"{output_dir}/image2.png"
with open(iter_image_path, "wb") as image_file:
    image_file.write(image_data)

#img = Image.open(image_path)  # nosec
#img.show()


### Prelim. Results

In [ ]:
Image(filename=iter_image_path, width=300)

## Generate Marketing Material

In [ ]:
marketing1 = "Place this object on"

marks = ["a business desk", "a kitchen table", "a bathroom counter", "a living room table", "a coffee table", "a dining room table", "a shelf", "a bookcase", "a nightstand", "a dresser"]

i = -1
for mark in marks:
    i += 1
    m_image_path = f"{output_dir}/image2_{i}.png"
    if os.path.exists(m_image_path):
        print(f"File {m_image_path} already exists. Skipping...")
        continue
    prompt = f"{marketing1} {mark}."


    print(prompt)
    image_base_64 = edit_image(open(f"{output_dir}/image2.png", "rb").read(), open("mask2.png", "rb").read(), prompt)  


    image_data = base64.b64decode(image_base_64)


    
    with open(m_image_path, "wb") as image_file:
        image_file.write(image_data)
    
    display(Image(filename=m_image_path, width=300))

    